In [17]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy 
import pandas
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
import csv
import json
from collections import Counter
import statistics 
import plotly.graph_objects as go
import math

In [18]:
from itertools import combinations
from scipy.stats import ttest_ind
import decimal

# create a new context for this task
ctx = decimal.Context()

# 20 digits should be enough for everyone :D
ctx.prec = 20

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

In [19]:
with open('../../responseJudgement.json') as json_file:
    judgements = json.load(json_file)
    
with open('../../responseJudgement_sugg.json') as json_file:
    suggJudgements = json.load(json_file)

In [20]:
len(judgements)

15417

In [21]:
len(suggJudgements)

1014

In [22]:
cleanPath = '../../clean/'
settingName = 'Setting 4'

In [23]:
responseSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + context['userResponse'].strip()
                        
                        if(uniqueStr in responseSet):
                            continue
                        
                        else:
                            
                            foundMatches = [x for x in judgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                            foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]
                            
                            if(len(foundJudgement) > 0):
                                responseSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [24]:
len(responseSet)

5139

In [25]:
suggSet = {}

for root, dirs, files in os.walk(cleanPath):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    currFileName = data['currentFileName']
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        suggestions = context['suggestions']
                        
                        for sugg in suggestions:
                            uniqueStr = 'fileIndex=' + str(context['fileIndex']) + '&&contextLen=' + str(context['contextLen']) + '&&userResponse=' + sugg.strip()

                            if(uniqueStr in responseSet or uniqueStr in suggSet):
                                continue

                            else:

                                foundMatches = [x for x in suggJudgements if x['Input.contextLen'] == str(context['contextLen']) and x['Input.fileIndex'] == str(context['fileIndex'])]
                                foundJudgement = [x for x in foundMatches if x['Input.fileName'] == currFileName and x['Input.localIndex'] == str(context['localIndex'])]

                                if(len(foundJudgement) > 0):
                                    suggSet[uniqueStr] = foundJudgement
                        
                except Exception as e: print(e)

In [26]:
len(suggSet)

458

In [27]:
Qualtype = 'Logical'
metric = {
    'Answer.' + Qualtype +'.Strongly Disagree': 1,
    'Answer.' + Qualtype +'.Disagree': 2,
    'Answer.' + Qualtype +'.Neutral': 3,
    'Answer.' + Qualtype +'.Agree': 4,
    'Answer.' + Qualtype +'.Strongly Agree': 5
}

In [28]:
scoresHuman = []

for root, dirs, files in os.walk(cleanPath + 'Setting 1'):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scoresHuman.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [29]:
scoresBot = []

for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        suggestions = context['suggestions']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + suggestions[0].strip()
                        
                        if(identifierStr in responseSet):
                            responseJudgements =  responseSet[identifierStr]
                        else:
                            responseJudgements =  suggSet[identifierStr]
                        
                        currFeedback = []
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        scoresBot.append(statistics.mean(currFeedback))
                            
                        
                    
                except Exception as e: print(e)                 

In [30]:
sugg1 = []
sugg3 = []
sugg5 = []

autoSugg1 = []
autoSugg3 = []
autoSugg5 = []


for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        contextLen = context['contextLen']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + userResponse.strip()
                        
                        responseJudgements =  responseSet[identifierStr]
                        
                        currFeedback = []
                        
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        
                        if(data['autoFilledSuggestion'] == 0):
                            
                            if(data['suggestionsNumber'] == 1):
                                sugg1.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                sugg3.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 5):
                                sugg5.append(statistics.mean(currFeedback))
                        
                        else:

                            if(data['suggestionsNumber'] == 1):
                                autoSugg1.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                autoSugg3.append(statistics.mean(currFeedback))
                            
                            elif(data['suggestionsNumber'] == 5):
                                autoSugg5.append(statistics.mean(currFeedback))
                    
                    
                except Exception as e: print(e)

In [31]:
sugg1Bot = []
sugg3Bot = []
sugg5Bot = []

autoSugg1Bot = []
autoSugg3Bot = []
autoSugg5Bot = []


for root, dirs, files in os.walk(cleanPath + settingName):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                try:
                    data = json.load(json_file)
                    dialogs = data['dialogs']
                    
                    for i in range(1, len(dialogs)):
                        context = dialogs[i]['context']
                        
                        userResponse = context['userResponse']
                        fileIndex = context['fileIndex']
                        suggestions = context['suggestions']
                        
                        identifierStr = 'fileIndex=' + str(fileIndex) + '&&contextLen=' + str(contextLen) + '&&userResponse=' + suggestions[0].strip()
                        
                        if(identifierStr in responseSet):
                            responseJudgements =  responseSet[identifierStr]
                        else:
                            responseJudgements =  suggSet[identifierStr]
                        
                        currFeedback = []
                        
                        for feedback in responseJudgements:
                            for key in feedback:
                                
                                if('Answer.' + Qualtype in key and feedback[key] == 'true'):
                                    currFeedback.append(metric[key])
                        
                        
                        if(data['autoFilledSuggestion'] == 0):
                            
                            if(data['suggestionsNumber'] == 1):
                                sugg1Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                sugg3Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 5):
                                sugg5Bot.append(statistics.mean(currFeedback))
                        
                        else:

                            if(data['suggestionsNumber'] == 1):
                                autoSugg1Bot.append(statistics.mean(currFeedback))

                            elif(data['suggestionsNumber'] == 3):
                                autoSugg3Bot.append(statistics.mean(currFeedback))
                            
                            elif(data['suggestionsNumber'] == 5):
                                autoSugg5Bot.append(statistics.mean(currFeedback))
                    
                    
                except Exception as e: print(e)

'fileIndex=5&&contextLen=3&&userResponse=Where were cakes first invented?'
'fileIndex=1&&contextLen=3&&userResponse=Yes I do, do you know a lot about it?'
'fileIndex=10&&contextLen=3&&userResponse=Yeah I bet it would be interesting, you are at the mercy of the waves. Any particular beaches you like?'
'fileIndex=12&&contextLen=3&&userResponse=My work does free blood testing every year, and last year mine showed I was pre-diabetic. I have tried to eat fewer cars as a result.'
"fileIndex=15&&contextLen=3&&userResponse=Yes she definitely has made quite the stir. As I recall her name was mile Stewart on that show and it's to bad she was not the teen idol she could have been."
'fileIndex=20&&contextLen=3&&userResponse=There is a competition called the Pan American Race Walking Cup that is held twice a year. Lol'
"fileIndex=26&&contextLen=3&&userResponse=Yeah! The first mention of the word in English however only dates back to the 30's"
"fileIndex=11&&contextLen=3&&userResponse=Which one is b

'fileIndex=1&&contextLen=3&&userResponse=Yes I do, do you know a lot about it?'
'fileIndex=24&&contextLen=3&&userResponse=It helps that the Association of Professional Pierces has developed safety standards.'
'fileIndex=5&&contextLen=3&&userResponse=Where were cakes first invented?'
'fileIndex=3&&contextLen=3&&userResponse=The English word is borrowed from Modern Greek πίτα. And a rough translation for that is bread cake! Pretty interesting huh?'
'fileIndex=16&&contextLen=3&&userResponse=Yeah I think dance is one of the most beautiful things humans can express, choreography is like a representation of emotion.'
'fileIndex=24&&contextLen=3&&userResponse=Yes! I love it so much, the TV series was created by David Sendoff and D. B. Weiss.'
'fileIndex=24&&contextLen=3&&userResponse=It helps that the Association of Professional Pierces has developed safety standards.'
'fileIndex=7&&contextLen=3&&userResponse=It is a very reliable car, so much that it is an official state car for transporting

In [32]:
settings = {
    'H': scoresHuman,
    'Bot': scoresBot,
    'sugg1': sugg1,
    'sugg3': sugg3,
    'sugg5': sugg5,
    'autoSugg1': autoSugg1,
    'autoSugg3': autoSugg3,
    'autoSugg5': autoSugg5
}

for list1, list2 in combinations(settings.keys(), 2):
    t, p = ttest_ind(settings[list1], settings[list2], equal_var = False)
    print (list1, list2, float_to_str(p))

H Bot 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006946147992997035
H sugg1 0.00000000000005861169359871057
H sugg3 0.0000000000006390479804168944
H sugg5 0.000000000994806832740109
H autoSugg1 0.0000000000000000000000000295670835016384
H autoSugg3 0.0000000000036166052569648478
H autoSugg5 0.00000000000000000000000000010327475142504875
Bot sugg1 0.00000000000000000000000000000000013465205892058472
Bot sugg3 0.0000000000000000000000000000000000009402524192505081
Bot sugg5 0.00000000000000000000000000000000000000019270212537749686
Bot autoSugg1 0.00000000000000000000000000005188372838422619
Bot autoSugg3 0.000000000000000000000000000000000000022106908326357268
Bot autoSugg5 0.0000000000000000000018137210946996862
sugg1 sugg3 0.7290127333671985
sugg1 sugg5 0.2051727881300386
sugg1 autoSugg1 0.021890706548949028
sugg1 autoSugg3 0.5533396997050133
sugg1 autoSugg5 0.0019971549716232646
sugg3 sugg5 0.35215553344863026
sugg3 auto

In [52]:
colors = ['#FAEBD7', '#FFA07A', '#B0C4DE', '#AFEEEE', '#8FBC8F']
x = ['1 Suggestion', '3 Suggestions', '5 Suggestions']
fig = go.Figure()

xHuman = ['']
yHuman = [scoresHuman]
yHumanErorr = [np.std(z)/math.sqrt(len(z)) for z in yHuman] 
yHuman = [round(statistics.mean(z), 2) for z in yHuman]

fig.add_trace(go.Bar(
    name='Human Only',
    x=xHuman, y=yHuman, text=yHuman, textposition='inside',insidetextanchor='middle',
    marker_color='#B0C4DE',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yHumanErorr)
))

xBot = ['']
yBot = [scoresBot]
yBotErorr = [np.std(z)/math.sqrt(len(z)) for z in yBot] 
yBot = [round(statistics.mean(z), 2) for z in yBot]

fig.add_trace(go.Bar(
    name='Chatbot Only',
    x=xBot, y=yBot, text=yBot, textposition='inside',insidetextanchor='middle',
    marker_color='#A9A9A9',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yBotErorr)
))


ySuggOnly = [sugg1, sugg3, sugg5]
ySuggOnlyErorr = [np.std(z)/math.sqrt(len(z)) for z in ySuggOnly] 
ySuggOnly = [round(statistics.mean(z), 2) for z in ySuggOnly]

fig.add_trace(go.Bar(
    name='C1',
    x=x, y=ySuggOnly, text=ySuggOnly, textposition='inside',insidetextanchor='middle',
    marker_color='lightsalmon',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=ySuggOnlyErorr)
))

ySuggOnlyBot = [sugg1Bot, sugg3Bot, sugg5Bot]
ySuggOnlyBotErorr = [np.std(z)/math.sqrt(len(z)) for z in ySuggOnlyBot] 
ySuggOnlyBot = [round(statistics.mean(z), 2) for z in ySuggOnlyBot]

fig.add_trace(go.Bar(
    name='C1 - Chatbot Only',
    x=x, y=ySuggOnlyBot, text=ySuggOnlyBot, textposition='inside',insidetextanchor='middle',
    marker_color='#ffdacc',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=ySuggOnlyBotErorr)
))


yAutoSugg = [autoSugg1, autoSugg3, autoSugg5]
yAutoSuggErorr = [np.std(z)/math.sqrt(len(z)) for z in yAutoSugg] 
yAutoSugg = [round(statistics.mean(z), 2) for z in yAutoSugg]

fig.add_trace(go.Bar(
    name='C2',
    x=x, y=yAutoSugg, text=yAutoSugg, textposition='inside',insidetextanchor='middle',
    marker_color='#AFEEEE',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yAutoSuggErorr)
))


yAutoSuggBot = [autoSugg1Bot, autoSugg3Bot, autoSugg5Bot]
yAutoSuggBotErorr = [np.std(z)/math.sqrt(len(z)) for z in yAutoSuggBot] 
yAutoSuggBot = [round(statistics.mean(z), 2) for z in yAutoSuggBot]

fig.add_trace(go.Bar(
    name='C2 - Chatbot Only',
    x=x, y=yAutoSuggBot, text=yAutoSuggBot, textposition='inside',insidetextanchor='middle',
    marker_color='#eafbfb',
    marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.8,
    error_y=dict(type='data', array=yAutoSuggBotErorr)
))


fig.update_layout(
    font=dict(
        size=13
    )
)
fig.update_yaxes(range=[2.4, 4])

fig.update_layout(legend_orientation="h")
fig.show()